<a href="https://colab.research.google.com/github/rvandergraaff/CURSO-DE-IA-2025/blob/main/thumbnails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import fitz  # PyMuPDF
from PIL import Image
import io
import os

def generar_thumbnails_desde_carpeta(carpeta_entrada_pdfs, tamaño_thumbnail=(128, 128), carpeta_salida="miniaturas_generadas"):
    """
    Recorre una carpeta de entrada, procesa todos los archivos PDF encontrados
    y genera una miniatura JPG de la primera imagen de cada PDF.

    Args:
        carpeta_entrada_pdfs (str): La ruta a la carpeta que contiene los archivos PDF.
        tamaño_thumbnail (tuple): Una tupla (ancho, alto) para el tamaño de la miniatura.
        carpeta_salida (str): El nombre de la carpeta donde se guardarán las miniaturas.
    """

    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)
        print(f"Carpeta de salida '{carpeta_salida}' creada en: {os.path.abspath(carpeta_salida)}")

    if not os.path.exists(carpeta_entrada_pdfs):
        print(f"Error: La carpeta de entrada '{carpeta_entrada_pdfs}' no existe. Asegúrate de que los PDFs estén allí.")
        return

    print(f"\n--- Buscando PDFs en la carpeta: {carpeta_entrada_pdfs} ---")
    archivos_encontrados = [f for f in os.listdir(carpeta_entrada_pdfs) if f.lower().endswith('.pdf')]

    if not archivos_encontrados:
        print(f"No se encontraron archivos PDF en la carpeta '{carpeta_entrada_pdfs}'.")
        return

    for nombre_pdf in archivos_encontrados:
        ruta_pdf_completa = os.path.join(carpeta_entrada_pdfs, nombre_pdf)
        print(f"\n--- Procesando PDF: {ruta_pdf_completa} ---")

        try:
            documento = fitz.open(ruta_pdf_completa)
            nombre_base_pdf = os.path.splitext(os.path.basename(ruta_pdf_completa))[0]

            imagen_encontrada_para_pdf = False
            for num_pagina in range(len(documento)):
                pagina = documento.load_page(num_pagina)
                imagenes_en_pagina = pagina.get_images(full=True)

                if imagenes_en_pagina:
                    print(f"  Página {num_pagina + 1}: Encontrada(s) imagen(es).")
                    # Tomamos la primera imagen de la página
                    img_info = imagenes_en_pagina[0]
                    xref = img_info[0] # XREF de la imagen
                    base_imagen = documento.extract_image(xref)
                    imagen_bytes = base_imagen["image"]

                    try:
                        # Abrir la imagen con Pillow
                        imagen = Image.open(io.BytesIO(imagen_bytes))

                        # Asegurarse de que la imagen está en un modo compatible para JPG (RGB o L)
                        if imagen.mode in ('RGBA', 'P', 'CMYK'):
                            imagen = imagen.convert('RGB')

                        # Redimensionar la imagen para el thumbnail
                        imagen.thumbnail(tamaño_thumbnail)

                        # Guardar la miniatura como JPG
                        nombre_archivo_thumbnail = f"{nombre_base_pdf}-thn.jpg"
                        ruta_salida_thumbnail = os.path.join(carpeta_salida, nombre_archivo_thumbnail)
                        imagen.save(ruta_salida_thumbnail, "JPEG") # Especificamos "JPEG"
                        print(f"    Miniatura guardada: {ruta_salida_thumbnail}")
                        imagen_encontrada_para_pdf = True
                        break # Ya encontramos y procesamos una imagen para este PDF, pasamos al siguiente PDF
                    except Exception as e:
                        print(f"    Error al procesar la primera imagen de la página {num_pagina + 1} de '{nombre_pdf}': {e}")
                # else:  # Opcional: comentar si no quieres ver el mensaje para páginas sin imágenes
                #     print(f"  Página {num_pagina + 1}: No se encontraron imágenes.")

            if not imagen_encontrada_para_pdf:
                print(f"  No se encontró ninguna imagen en ninguna página de '{nombre_pdf}'. No se generó miniatura.")

        except fitz.FileNotFoundError:
            print(f"Error: El archivo PDF '{ruta_pdf_completa}' no fue encontrado. Esto no debería ocurrir si os.listdir funcionó bien, pero se registra por seguridad.")
        except Exception as e:
            print(f"Ocurrió un error inesperado al procesar '{ruta_pdf_completa}': {e}")
        finally:
            if 'documento' in locals() and documento:
                documento.close()
    print("\n--- Procesamiento de carpeta completado. ---")

# --- Ejemplo de uso en Google Colab ---
if __name__ == "__main__":
    # --- PASO 1: CREA UNA CARPETA EN COLAB Y SUBE TUS PDFs A ELLA ---
    # Por ejemplo, vamos a crear una carpeta llamada 'mis_pdfs' en el entorno de Colab.
    # Luego, DEBES SUBIR TUS ARCHIVOS PDF REALES a esa carpeta usando el panel de archivos de Colab.

    carpeta_donde_subiras_tus_pdfs = "mis_pdfs" # Puedes cambiar el nombre de la carpeta si lo deseas

    # Crea la carpeta en Colab si no existe.
    if not os.path.exists(carpeta_donde_subiras_tus_pdfs):
        os.makedirs(carpeta_donde_subiras_tus_pdfs)
        print(f"Carpeta '{carpeta_donde_subiras_tus_pdfs}' creada en Colab. Por favor, sube tus PDFs aquí.")

    # --- (Opcional) Crea algunos PDFs de ejemplo si no tienes para probar ---
    # Esto es solo para que el script tenga algo con qué trabajar si estás probando
    # y no has subido tus propios PDFs aún.
    # Si subes tus propios PDFs, puedes comentar o eliminar este bloque.
    print("\n(Opcional) Creando PDFs de ejemplo para demostración. Si ya subiste tus PDFs, puedes ignorar esto.")
    for i in range(1, 4): # Crea 3 PDFs de ejemplo
        pdf_ejemplo_nombre = f"documento_ejemplo_{i}.pdf"
        ruta_ejemplo_pdf = os.path.join(carpeta_donde_subiras_tus_pdfs, pdf_ejemplo_nombre)
        if not os.path.exists(ruta_ejemplo_pdf):
            doc = fitz.open()
            page = doc.new_page()
            page.insert_text((50, 50), f"Este es el contenido del {pdf_ejemplo_nombre}.", fontname="helv", fontsize=12)
            # Para que tenga una imagen real y se genere thumbnail, puedes insertar una imagen
            # Por ejemplo: page.insert_image(fitz.Rect(100, 100, 200, 200), filename="ruta_a_tu_imagen.png")
            # Sin una imagen, el script dirá "No se encontró ninguna imagen..."
            doc.save(ruta_ejemplo_pdf)
            doc.close()
            print(f"  PDF de ejemplo '{pdf_ejemplo_nombre}' creado en '{carpeta_donde_subiras_tus_pdfs}'.")
        else:
            print(f"  PDF de ejemplo '{pdf_ejemplo_nombre}' ya existe.")
    # --- Fin del bloque de PDFs de ejemplo ---

    # --- PASO 2: LLAMA A LA FUNCIÓN PRINCIPAL ---
    # Aquí es donde le dices al script qué carpeta debe escanear.
    generar_thumbnails_desde_carpeta(carpeta_donde_subiras_tus_pdfs, tamaño_thumbnail=(250, 250), carpeta_salida="miniaturas_finales")

    # --- PASO 3: DESCARGA LOS THUMBNAILS DESDE COLAB (Opcional) ---
    # Una vez que el script termine, puedes ejecutar esto en una nueva celda
    # para comprimir y descargar todas las miniaturas generadas.
    import shutil
    shutil.make_archive("miniaturas_finales", 'zip', "miniaturas_finales")
    from google.colab import files
    files.download("miniaturas_finales.zip")


(Opcional) Creando PDFs de ejemplo para demostración. Si ya subiste tus PDFs, puedes ignorar esto.
  PDF de ejemplo 'documento_ejemplo_1.pdf' ya existe.
  PDF de ejemplo 'documento_ejemplo_2.pdf' ya existe.
  PDF de ejemplo 'documento_ejemplo_3.pdf' ya existe.

--- Buscando PDFs en la carpeta: mis_pdfs ---

--- Procesando PDF: mis_pdfs/(237)MATEMATICA-51-PRESENCIAL-UNICA-1P-2025.pdf ---
  Página 1: Encontrada(s) imagen(es).
    Miniatura guardada: miniaturas_finales/(237)MATEMATICA-51-PRESENCIAL-UNICA-1P-2025-thn.jpg

--- Procesando PDF: mis_pdfs/(239)MATEMATICA-51-PRESENCIAL-ROSSOMANDO-1P-2025.pdf ---
  Página 1: Encontrada(s) imagen(es).
    Miniatura guardada: miniaturas_finales/(239)MATEMATICA-51-PRESENCIAL-ROSSOMANDO-1P-2025-thn.jpg

--- Procesando PDF: mis_pdfs/(240)BIOFISICA-53-PRESENCIAL-SILVA-1P-2025.pdf ---
  Página 1: Encontrada(s) imagen(es).
    Miniatura guardada: miniaturas_finales/(240)BIOFISICA-53-PRESENCIAL-SILVA-1P-2025-thn.jpg

--- Procesando PDF: mis_pdfs/document

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# New Section